# One-to-Many and Many-to-Many Joins

## Introduction

Previously, you learned about the typical case where one joins on a primary or foreign key. In this section, you'll explore other types of joins using one-to-many and many-to-many relationships!

## Objectives

You will be able to:

* Explain one-to-many and many-to-many joins as well as implications for the size of query results
* Query data using one-to-many and many-to-many joins

## One-to-Many and Many-to-Many Relationships

So far, you've seen a couple of different kinds of join statements: `LEFT JOIN` and `INNER JOIN` (aka, `JOIN`). Both of these refer to the way in which you would like to define your join based on the tables and their shared information. Another perspective on this is the number of matches between the tables based on your defined links with the keywords `ON` or `USING`.
  
You have also seen the typical case where one joins on a primary or foreign key. For example, when you join on `customerID` or `employeeID`, this value should be unique to that table. As such, your joins have been very similar to using a dictionary to find additional information associated with that record. In cases where there are multiple entries in either table for the field (column) you are joining on, you will similarly be given multiple rows in your resulting view, one for each of these entries.

### Restaurants Case Study

We'll start with this familiar ERD:

<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width=550>

For example, let's say you have another table `restaurants` that has many columns including `name`, `city`, and `rating`. If you were to join this `restaurants` table with the offices table using the shared city column, you might get some unexpected behavior. That is, in the office table, there is only one office per city. However, because there will likely be more than one restaurant for each of these cities in your second table, you will get unique combinations of offices and restaurants from your join. If there are 513 restaurants for Boston in your restaurant table and 1 office for Boston, your joined table will have each of these 513 rows, one for each restaurant along with the one office.

If you had 2 offices for Boston and 513 restaurants, your join would have 1026 rows for Boston; 513 for each restaurant along with the first office and 513 for each restaurant with the second office. Three offices in Boston would similarly produce 1539 rows; one for each unique combination of restaurants and offices. This is where you should be particularly careful of many to many joins as the resulting set size can explode drastically, potentially consuming vast amounts of memory and other resources.  

## Connecting to the Database

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('data.sqlite')

## One-to-One Relationships

Sometimes, a `JOIN` does not increase the number of records at all. For example, in our database, each employee is only associated with one office. So if our original query included information about all employees with a `jobTitle` of "Sales Rep", then our joined query also added the city location of their offices, we would get the same number of results both times.

### Sales Rep Employees

In [3]:
q = """
SELECT firstName, lastName, email
FROM employees
WHERE jobTitle = 'Sales Rep'
;
"""
df = pd.read_sql(q, conn)
print("Number of results:", len(df))

Number of results: 17


In [4]:
df

,firstName,lastName,email
0,Leslie,Jennings,ljennings@classicmodelcars.com
1,Leslie,Thompson,lthompson@classicmodelcars.com
2,Julie,Firrelli,jfirrelli@classicmodelcars.com
3,Steve,Patterson,spatterson@classicmodelcars.com
4,Foon Yue,Tseng,ftseng@classicmodelcars.com
5,George,Vanauf,gvanauf@classicmodelcars.com
6,Loui,Bondur,lbondur@classicmodelcars.com
7,Gerard,Hernandez,ghernande@classicmodelcars.com
8,Pamela,Castillo,pcastillo@classicmodelcars.com
9,Larry,Bott,lbott@classicmodelcars.com


In [4]:
# Displaying only 5 for readability
df.head()

,firstName,lastName,email
0,Leslie,Jennings,ljennings@classicmodelcars.com
1,Leslie,Thompson,lthompson@classicmodelcars.com
2,Julie,Firrelli,jfirrelli@classicmodelcars.com
3,Steve,Patterson,spatterson@classicmodelcars.com
4,Foon Yue,Tseng,ftseng@classicmodelcars.com


### Cities for Sales Rep Employees

Now we'll join with the `offices` table in order to display the city name as well.

In [5]:
q = """
SELECT firstName, lastName, email, city
FROM employees
JOIN offices
    USING(officeCode)
WHERE jobTitle = 'Sales Rep'
;
"""
df = pd.read_sql(q, conn)
print("Number of results:", len(df))

Number of results: 17


In [6]:
# Displaying only 5 for readability
df.head()

,firstName,lastName,email,city
0,Leslie,Jennings,ljennings@classicmodelcars.com,San Francisco
1,Leslie,Thompson,lthompson@classicmodelcars.com,San Francisco
2,Julie,Firrelli,jfirrelli@classicmodelcars.com,Boston
3,Steve,Patterson,spatterson@classicmodelcars.com,Boston
4,Foon Yue,Tseng,ftseng@classicmodelcars.com,NYC


As you can see, we got the same number of results as the original query, we just have more data now.

## One-to-Many Relationships

When there is a one-to-many relationship, the number of records will increase to match the number of records in the larger table.

### Product Lines

Let's start with selecting the product line name and text description for all product lines.

In [8]:
pd.read_sql(""" SELECT * FROM productlines; """,conn)

,productLine,textDescription,htmlDescription,image
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,,
1,Motorcycles,Our motorcycles are state of the art replicas ...,,
2,Planes,"Unique, diecast airplane and helicopter replic...",,
3,Ships,The perfect holiday or anniversary gift for ex...,,
4,Trains,Model trains are a rewarding hobby for enthusi...,,
5,Trucks and Buses,The Truck and Bus models are realistic replica...,,
6,Vintage Cars,Our Vintage Car models realistically portray a...,,


In [9]:
q = """
SELECT productLine, textDescription
FROM productlines
;
"""
df = pd.read_sql(q, conn)
print("Number of results:", len(df))

Number of results: 7


In [8]:
df

,productLine,textDescription
0,Classic Cars,Attention car enthusiasts: Make your wildest c...
1,Motorcycles,Our motorcycles are state of the art replicas ...
2,Planes,"Unique, diecast airplane and helicopter replic..."
3,Ships,The perfect holiday or anniversary gift for ex...
4,Trains,Model trains are a rewarding hobby for enthusi...
5,Trucks and Buses,The Truck and Bus models are realistic replica...
6,Vintage Cars,Our Vintage Car models realistically portray a...


### Joining with Products

Now let's join that table with the products table, and select the vendor and product description.

In [11]:
pd.read_sql(""" SELECT * FROM products;""",conn).head()

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00


In [22]:
q= '''
SELECT pl.productLine, pl.textdescription, pr.productvendor,pr.productdescription
FROM productlines AS pl
JOIN products AS pr
ON pl.productLine=pr.productLine
WHERE pl.productline="Classic Cars";
'''
pd.read_sql(q,conn)

,productLine,textDescription,productVendor,productDescription
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,Autoart Studio Design,"Hood, doors and trunk all open to reveal highl..."
1,Classic Cars,Attention car enthusiasts: Make your wildest c...,Carousel DieCast Legends,Features include opening and closing doors. Co...
2,Classic Cars,Attention car enthusiasts: Make your wildest c...,Carousel DieCast Legends,The operating parts of this 1958 Chevy Corvett...
3,Classic Cars,Attention car enthusiasts: Make your wildest c...,Carousel DieCast Legends,This diecast model of the 1966 Shelby Cobra 42...
4,Classic Cars,Attention car enthusiasts: Make your wildest c...,Classic Metal Creations,1957 die cast Corvette Convertible in Roman Re...
5,Classic Cars,Attention car enthusiasts: Make your wildest c...,Classic Metal Creations,Die-cast model of the silver 1965 Aston Martin...
6,Classic Cars,Attention car enthusiasts: Make your wildest c...,Classic Metal Creations,Features include: Turnable front wheels; steer...
7,Classic Cars,Attention car enthusiasts: Make your wildest c...,Classic Metal Creations,Precision-engineered from original Jaguar spec...
8,Classic Cars,Attention car enthusiasts: Make your wildest c...,Classic Metal Creations,This 1:18 scale precision die-cast reproductio...
9,Classic Cars,Attention car enthusiasts: Make your wildest c...,Classic Metal Creations,Turnable front wheels; steering function; deta...


In [23]:
q= '''
SELECT pl.productLine, pl.textdescription, pr.productvendor,pr.productdescription
FROM productlines AS pl
JOIN products AS pr
ON pl.productLine=pr.productLine;
'''
df=pd.read_sql(q,conn)
print("Number of results:", len(df))


Number of results: 110


In [24]:
q = """
SELECT productLine, textDescription, productVendor, productDescription
FROM productlines
JOIN products
    USING(productLine)
;
"""
df = pd.read_sql(q, conn)
print("Number of results:", len(df))

Number of results: 110


In [25]:
# Displaying only 5 for readability
df.head()

,productLine,textDescription,productVendor,productDescription
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,Autoart Studio Design,"Hood, doors and trunk all open to reveal highl..."
1,Classic Cars,Attention car enthusiasts: Make your wildest c...,Carousel DieCast Legends,Features include opening and closing doors. Co...
2,Classic Cars,Attention car enthusiasts: Make your wildest c...,Carousel DieCast Legends,The operating parts of this 1958 Chevy Corvett...
3,Classic Cars,Attention car enthusiasts: Make your wildest c...,Carousel DieCast Legends,This diecast model of the 1966 Shelby Cobra 42...
4,Classic Cars,Attention car enthusiasts: Make your wildest c...,Classic Metal Creations,1957 die cast Corvette Convertible in Roman Re...


As you can see, the number of records has increased significantly, because the same product line is now appearing multiple times in the results, once for each actual product.

## Many-to-Many Relationships

A many-to-many join is as it sounds; there are multiple entries for the shared field in both tables. While somewhat contrived, we can see this through the example below, joining the offices and customers table based on the state field. For example, there are 2 offices in MA and 9 customers in MA. Joining the two tables by state will result in 18 rows associated with MA; one for each customer combined with the first office, and then another for each customer combined with the second option. This is not a particularly useful join without applying some additional aggregations or pivots, but can also demonstrate how a poorly written query can go wrong. For example, if there are a large number of occurrences in both tables, such as tens of thousands, then a many-to-many join could result in billions of resulting rows. Poorly conceived joins can cause a severe load to be put on the database, causing slow execution time and potentially even tying up database resources for other analysts who may be using the system.

### Just Offices

In [30]:
q='''
SELECT *
FROM offices;
''' 
df=pd.read_sql(q,conn)
df


,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan
5,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
6,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA


In [28]:
q = """
SELECT *
FROM offices
;
"""

df = pd.read_sql(q, conn)
print('Number of results:', len(df))

Number of results: 7


### Just Customers

In [31]:
q="""
SELECT *
FROM customers
;
"""
pd.read_sql(q,conn)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",,Nantes,,44000,France,1370,21000.00
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,,Las Vegas,NV,83030,USA,1166,71800.00
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611,117300.00
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",,Nantes,,44000,France,1370,118200.00
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,,Stavern,,4110,Norway,1504,81700.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,486,Motor Mint Distributors Inc.,Salazar,Rosa,2155559857,11328 Douglas Av.,,Philadelphia,PA,71270,USA,1323,72600.00
118,487,Signal Collectibles Ltd.,Taylor,Sue,4155554312,2793 Furth Circle,,Brisbane,CA,94217,USA,1165,60300.00
119,489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,120 Hanover Sq.,,London,,WA1 1DP,UK,1501,43300.00
120,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,,Boston,MA,51003,USA,1188,85100.00


In [12]:
q = """
SELECT *
FROM customers
;
"""

df = pd.read_sql(q, conn)
print('Number of results:', len(df))

Number of results: 122


### Joined Offices and Customers

In [33]:
q= '''
SELECT *
FROM offices
JOIN customers
    USING (state)
    ;
'''
pd.read_sql(q,conn)

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory,customerNumber,...,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,124,...,Nelson,Susan,4155551450,5677 Strong St.,,San Rafael,97562,USA,1165,210500.00
1,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,129,...,Murphy,Julie,6505555787,5557 North Pendale Street,,San Francisco,94217,USA,1165,64600.00
2,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,161,...,Hashimoto,Juri,6505556809,9408 Furth Circle,,Burlingame,94217,USA,1165,84600.00
3,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,205,...,Young,Julie,6265557265,78934 Hillside Dr.,,Pasadena,90003,USA,1166,90700.00
4,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,219,...,Young,Mary,3105552373,4097 Douglas Av.,,Glendale,92561,USA,1166,11000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA,480,...,Semenov,Alexander,+7 812 293 0521,2 Pobedy Square,,Saint Petersburg,196143,Russia,,0.00
241,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA,481,...,"Altagar,G M",Raanan,+ 972 9 959 8555,3 Hagalim Blv.,,Herzlia,47625,Israel,,0.00
242,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA,484,...,Roel,José Pedro,(95) 555 82 82,"C/ Romero, 33",,Sevilla,41101,Spain,1702,65700.00
243,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA,489,...,Smith,Thomas,(171) 555-7555,120 Hanover Sq.,,London,WA1 1DP,UK,1501,43300.00


In [32]:
q = """
SELECT *
FROM offices
JOIN customers
    USING(state)
;
"""

df = pd.read_sql(q, conn)
print('Number of results:', len(df))

Number of results: 245


In [14]:
# Displaying only 5 for readability
df.head()

,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory,customerNumber,...,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,postalCode,country,salesRepEmployeeNumber,creditLimit
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,124,...,Nelson,Susan,4155551450,5677 Strong St.,,San Rafael,97562,USA,1165,210500
1,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,129,...,Murphy,Julie,6505555787,5557 North Pendale Street,,San Francisco,94217,USA,1165,64600
2,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,161,...,Hashimoto,Juri,6505556809,9408 Furth Circle,,Burlingame,94217,USA,1165,84600
3,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,205,...,Young,Julie,6265557265,78934 Hillside Dr.,,Pasadena,90003,USA,1166,90700
4,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA,219,...,Young,Mary,3105552373,4097 Douglas Av.,,Glendale,92561,USA,1166,11000


Whenever you write a SQL query, make sure you understand the unit of analysis you are trying to use. Getting more data from the database is not always better! The above query might make sense as a starting point for something like "what is the ratio of customers to offices in each state", but it's not there yet. Many-to-many joins can be useful, but it's important to be strategic and understand what you're really asking for.

## Summary

In this section, you expanded your join knowledge to one-to-many and many-to-many joins!